In [1]:
from collections import OrderedDict
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
from torchvision.models import resnet101
import torch.nn as nn
import os
import cv2
import random
import torch.utils.model_zoo as model_zoo
from copy import deepcopy

from matplotlib import cm
import scipy.misc as misc
import matplotlib
import imageio

In [2]:
from collections import OrderedDict

import torch
from torch import nn


class IntermediateLayerGetter(nn.ModuleDict):
    """
    Module wrapper that returns intermediate layers from a model
    It has a strong assumption that the modules have been registered
    into the model in the same order as they are used.
    This means that one should **not** reuse the same nn.Module
    twice in the forward if you want this to work.
    Additionally, it is only able to query submodules that are directly
    assigned to the model. So if `model` is passed, `model.feature1` can
    be returned, but not `model.feature1.layer2`.
    Arguments:
        model (nn.Module): model on which we will extract the features
        return_layers (Dict[name, new_name]): a dict containing the names
            of the modules for which the activations will be returned as
            the key of the dict, and the value of the dict is the name
            of the returned activation (which the user can specify).
    Examples::
        >>> m = torchvision.models.resnet18(pretrained=True)
        >>> # extract layer1 and layer3, giving as names `feat1` and feat2`
        >>> new_m = torchvision.models._utils.IntermediateLayerGetter(m,
        >>>     {'layer1': 'feat1', 'layer3': 'feat2'})
        >>> out = new_m(torch.rand(1, 3, 224, 224))
        >>> print([(k, v.shape) for k, v in out.items()])
        >>>     [('feat1', torch.Size([1, 64, 56, 56])),
        >>>      ('feat2', torch.Size([1, 256, 14, 14]))]
    """
    def __init__(self, model, return_layers):
        if not set(return_layers).issubset([name for name, _ in model.named_children()]):
            raise ValueError("return_layers are not present in model")

        orig_return_layers = return_layers
        return_layers = {k: v for k, v in return_layers.items()}
        layers = OrderedDict()
        for name, module in model.named_children():
            layers[name] = module
            if name in return_layers:
                del return_layers[name]
            if not return_layers:
                break

        super(IntermediateLayerGetter, self).__init__(layers)
        self.return_layers = orig_return_layers
        #self.softmax = nn.Softmax(dim=2)

    def forward(self, x):
        out = OrderedDict()
        for name, module in self.named_children():
            x = module(x)
            if name in self.return_layers:
                out_name = self.return_layers[name]
                out[out_name] = x#self.softmax(x)
        return out

In [3]:

class DoubleConv(nn.Module):#为U_Net模型中的双卷积结构
    def __init__(self, in_ch, out_ch):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),#此处包含padding，为了使输出图像与原图像大小相同
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, input):
        return self.conv(input)


class Unet(nn.Module):
    def __init__(self,in_ch,out_ch):
        super(Unet, self).__init__()

        self.conv1 = DoubleConv(in_ch, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.conv4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(2)
        self.conv5 = DoubleConv(512, 1024)
        self.up6 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.conv6 = DoubleConv(1024, 512)
        self.up7 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.conv7 = DoubleConv(512, 256)
        self.up8 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.conv8 = DoubleConv(256, 128)
        self.up9 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.conv9 = DoubleConv(128, 64)
        self.conv10 = nn.Conv2d(64,out_ch, 1)

    def forward(self,x):
        c1=self.conv1(x)
        p1=self.pool1(c1)
        c2=self.conv2(p1)
        p2=self.pool2(c2)
        c3=self.conv3(p2)
        p3=self.pool3(c3)
        c4=self.conv4(p3)
        p4=self.pool4(c4)
        c5=self.conv5(p4)
        up_6= self.up6(c5)
        merge6 = torch.cat([up_6, c4], dim=1)
        c6=self.conv6(merge6)
        up_7=self.up7(c6)
        merge7 = torch.cat([up_7, c3], dim=1)
        c7=self.conv7(merge7)
        up_8=self.up8(c7)
        merge8 = torch.cat([up_8, c2], dim=1)
        c8=self.conv8(merge8)
        up_9=self.up9(c8)
        merge9=torch.cat([up_9,c1],dim=1)
        c9=self.conv9(merge9)
        c10=self.conv10(c9)
        out = nn.Sigmoid()(c10)
        return out

model=Unet(1,1).cuda()

In [4]:

#criterion = nn.CrossEntropyLoss()
# model.load_state_dict(torch.load("deep3_(4_8).pkl"))
#criterion = nn.BCELoss()
learning_rate=0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
import csv,os
def get_csv():
    class_path  =  '...\\Data\\img\\'
    class_path3  = '...\\2.deeplab\\gen_t3\\'
    blanket=set()
    blanket2={}

    #############case3
    blanket.add(6)
    blanket.add(21)
    blanket.add(27)
#################case1
#     blanket.add(8)
#     blanket.add(9)
#     blanket.add(2)
###############case2
#     blanket.add(5)
#     blanket.add(15)
#     blanket.add(18)
###############case4
#     blanket.add(7)
#     blanket.add(29)
#     blanket.add(34)


#     for img_name in os.listdir(class_path):
#         info=str.split(img_name,"_")
#         blanket2[info[1]]=info[0]
#     for key in blanket2:
#         print(key+':'+blanket2[key])

#     return
    with open('case3_before_o.csv','w',newline='')as f:
        f_csv = csv.writer(f)
        i=0
        for img_name in os.listdir(class_path3):
            if img_name[0]!='_':
                continue
            info=str.split(img_name[1:],"_")
#             print(info)
            info3=info[0]
#             print(info3) #path
            info2=str.split(info[1],".")
            info2=info2[0]+".bmp"
#             print(info2) #file
            img_path2= class_path + info3+'\\'+info2
            img_path = class_path3+img_name
            temp=[2]
            temp.append(img_path2)
            img_name2=str.split(img_name[1:],".")
            img_name2=img_name2[0]+'.bmp'
            temp.append(img_name2)
            temp.append(img_path)
            temp.append("")
            f_csv.writerow(temp)
#             break
            i=i+1
            if i%100==0:
                print(i)
#             break

get_csv()


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [6]:
#keep the X contain the original image 
def load_data(label_path):
    labels_list=open(label_path, 'r')
    X,Y,Z,W=[],[],[],[]
    i=1
    c=0
    d=0
    for line in labels_list:
        info=str.split(line,",") 
        if i==1:
            print(info[1])
            print(info[3])
        X.append(np.expand_dims(imageio.imread(info[1]),axis=2))#orginal image
#         a=imageio.imread(info[3])
        W.append(np.expand_dims(imageio.imread(info[3]),axis=2))
#         b=np.max(a)
#         if b!=0:
#             b=1
#         if b:
#             d+=1
#         else:
#             c+=1
        Y.append(2)
        Z.append(str(info[2]))
        #break
#         print(X[0].shape)
#         print(X[0].shape)
        i+=1
        if i%100==0:
            print(i)
#     print(c,d)
    return list(zip(X,Y,Z,W))

In [8]:

# data=load_data("/media/students/000FF23E00059C2D/tq/2019/8/20/gen_test2.csv")
# data=load_data2("case4_before_o.csv")
# data=load_data3("my_test2.csv")
# data=load_data("unet_train2.csv")
data=load_data("case3_before_o.csv")

F:\Ruiveen2\Data\img\10072\1.bmp
F:\tq\2020\5\23\王铁桥-毕业论文-程序\2.deeplab\gen_t3\_10072_1.bmp
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [9]:
# model.load_state_dict(torch.load("deep1_o.pkl"))
mmpp=87685435454 # 1194110
model.load_state_dict(torch.load("unet3_o.pkl"))

<All keys matched successfully>

In [10]:

# #prepare data W{}
# with torch.no_grad():
#     batch_size=2
#     num_epochs=1
#     for epoch in range(num_epochs):
#         random.shuffle(data)
#         X, Y,_,W= zip(*data)
#         X=np.array(X).transpose((0,3,1,2))
#         W=np.array(W).transpose((0,3,1,2))
#         Y=np.array(Y)
#         for i in range(0,X.shape[0],batch_size):
#             if i+batch_size>=X.shape[0]: break
#             batch=torch.from_numpy(X[i:i+batch_size]).cuda().float()
#             outputs=model(batch)
#             outputs2=outputs.cpu().detach()
#             if i%100==0:
#                 print(i)
#             for j in range(batch_size):
#                 if Y[i+j]==1:
#                     W[i+j]=outputs2[j]

In [ ]:
import random #训练
batch_size=2
num_epochs=100
criterion = nn.CrossEntropyLoss()#torch.tensor([1,512]).float().cuda())
criterion2 = nn.BCELoss()

m=nn.Softmax(dim=1)
for epoch in range(num_epochs):
    count=0
    random.shuffle(data)
    X, Y,_,W= zip(*data)
    X=np.array(X).transpose((0,3,1,2))
    W=np.array(W).transpose((0,3,1,2))
    #print(W.shape)
    mm=0
    for i in range(0,X.shape[0],batch_size):
        if i+batch_size>=X.shape[0]: break
        batch=torch.from_numpy(X[i:i+batch_size]).cuda().float()
        outputs=model(batch)
#         outputs2=outputs.cpu().detach()
        outputs=outputs.squeeze()
        labels=torch.from_numpy(W[i:i+batch_size]).cuda().float()
#         for j in range(batch_size):
#             if Y[i+j]==1:
#                 W[i+j]=outputs2[j]
#         print(labels.shape)
        labels=torch.clamp(labels,0,1)
        labels=labels.view(labels.shape[0],labels.shape[2],labels.shape[3])
        loss = criterion2(outputs, labels)
        o_,pred=torch.max(outputs,1)
        mm+=loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if count%100==0:
            print('Epoch [{}/{}], step[{}/{}],Loss: {:.20f}'.format(epoch + 1, num_epochs, count,X.shape[0]//batch_size,loss.item()))
            print(mm)
        count += 1
    if mm<mmpp:
        mmpp=mm
        torch.save(model.state_dict(),"unet5_o.pkl")
#         learning_rate=min(0.02,learning_rate+0.0001)
#         optimizer = torch.optim.Ada m(model.parameters(), lr=learning_rate)
    else:
        torch.save(model.state_dict(),"unet5_o.pkl")
#         learning_rate=0.0001
#         optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    print(mm)
    print(mmpp)

Epoch [1/100], step[0/1117],Loss: 0.00087432190775871277
0.0008743219077587128


In [ ]:
# print(data)